<a href="https://colab.research.google.com/github/maximecharriere/CryptoWallet/blob/main/CryptoWallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Import libraries


In [ ]:
import os
from CryptoWallet.Loader import BinanceLoader, SwissborgLoader, KucoinLoader, BybitLoader, ManualTransactionsLoader, CoinbaseLoader
from CryptoWallet.Wallet import Wallet
import pandas as pd
user_home_dir = os.path.expanduser('~')

# Open database


In [ ]:
database_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\data\transactions.csv")

CRYPTOCOMPARE_API_KEY = os.getenv("CRYPTOCOMPARE_API_KEY")
if not CRYPTOCOMPARE_API_KEY:
    raise ValueError("CRYPTOCOMPARE_API_KEY is not set")

wallet = Wallet(apiKey=CRYPTOCOMPARE_API_KEY,
                databaseFilename=database_path)

## Import Binance transactions


In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\Binance")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(BinanceLoader.load(filepath))

wallet.save()

## Import Swissborg transactions


In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\SwissBorg")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(SwissborgLoader.load(filepath))

wallet.save()

## Import Kucoin transactions


In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\Kucoin")

for folder_name in os.listdir(transactions_path):
    # Create the full file path
    subfolder_path = os.path.join(transactions_path, folder_name)

    # Call your processing function
    wallet.addTransactions(KucoinLoader.load(subfolder_path))

wallet.save()

## Import Bybit transactions

In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\ByBit")

for folder_name in os.listdir(transactions_path):
    # Create the full file path
    subfolder_path = os.path.join(transactions_path, folder_name)

    # Call your processing function
    wallet.addTransactions(BybitLoader.load(subfolder_path))

wallet.save()

# Import Coinbase transactions

In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\Coinbase")

wallet_coinbase = Wallet(apiKey=CRYPTOCOMPARE_API_KEY,
                databaseFilename=os.path.dirname(database_path) + r"\transactions_coinbase.csv")

for folder_name in os.listdir(transactions_path):
    # Create the full file path
    subfolder_path = os.path.join(transactions_path, folder_name)

    # Call your processing function
    wallet_coinbase.addTransactions(CoinbaseLoader.load(subfolder_path))

wallet_coinbase.save()

print("WARNING: Please check the transactions and make sure they are correct. Loader not 100% accurate for Coinbase. Please check the transactions manually.")

## Import Manual Transactions files

In [ ]:
transactions_path = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\ExportedTransactions\Manual")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(ManualTransactionsLoader.load(filepath))

wallet.save()

## Export stats


In [ ]:
stats_filepath = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\data\stats.xlsx")

with pd.ExcelWriter(stats_filepath, engine='openpyxl') as writer:
    wallet.getStatsTot().to_excel(writer, sheet_name='StatsTot')
    wallet.getWalletsBalance().to_excel(writer, sheet_name='Wallets')
    transactions = wallet.transactions.copy()
    transactions['datetime'] = transactions['datetime'].dt.tz_localize(None)
    transactions.to_excel(writer, sheet_name='Transactions', index=False)

# Export for TradingView


In [ ]:
TradingView_filepath = os.path.join(
    user_home_dir, r"SynologyDrive\Documents\05_Crypto\data\transactionsToPlot.txt")

wallet.exportTradingView(TradingView_filepath)

# Save transactions


In [ ]:
wallet.save()